# Módulos útiles para ficheros grib

Para abrir el notebook en Colab pulsa el botón: <a target="_blank" href="https://colab.research.google.com/github/jlcasador/curso_formatos_meteo/blob/main/Modulos_para_grib.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[GRIB](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them) es un acrónimo cuyas siglas corresponden a "General Regularly distributed Information in Binary form".  
Es un formato estándar de la Organización Meteorológica Mundial (OMM) para archivar e intercambiar datos meteorológicos.

Esta foto puede representar la jerarquía de paquetes que vamos a usar en este curso:

![jerarquía de módulos](matroshka_grib_60.jpg)

## Instalación

### Instalación de eccodes utilizando CMake

In [ ]:
!wget https://confluence.ecmwf.int/download/attachments/45757960/eccodes-2.30.0-Source.tar.gz?api=v2 -O eccodes-2.30.0-Source.tar.gz
!tar -xzf  /content/eccodes-2.30.0-Source.tar.gz
!mkdir build ; cd build
%cd build
!cmake -DCMAKE_INSTALL_PREFIX=eccodes ../eccodes-2.30.0-Source
!make -j 4
!make install
%cd /content

In [ ]:
import os
PATH = os.getenv('PATH')
os.environ['PATH'] = PATH + ':/content/build/eccodes/bin'

In [ ]:
!wget https://github.com/waveology/tests/raw/main/local_grib_definitions.rar
!wget https://github.com/waveology/tests/raw/main/data.grib
!unrar x -o+ /content/local_grib_definitions.rar

In [ ]:
!codes_info

In [ ]:
os.environ['GRIB_DEFINITION_PATH'] = '/content/local_grib_definitions:/usr/local/share/grib_api/definitions'

### Instalación de eccodes con Conda

In [ ]:
!wget https://github.com/waveology/tests/raw/main/local_grib_definitions.rar
!wget https://github.com/waveology/tests/raw/main/data.grib
!unrar x -o+ /content/local_grib_definitions.rar && rm -rf /content/local_grib_definitions.rar

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
!conda install -q eccodes -c conda-forge

In [ ]:
!codes_info

In [ ]:
import os
os.environ['GRIB_DEFINITION_PATH'] = '/content/local_grib_definitions:/usr/local/share/grib_api/definitions'
!echo $GRIB_DEFINITION_PATH

### Instalación de climetlab

In [ ]:
!pip install climetlab

## Grib_tools

## Módulo eccodes

In [1]:
!ls -l

total 752
-rw-r--r-- 1 pn31 pn   8926 Jul  1 10:10 Descargas_MARS.ipynb
-rw-r--r-- 1 pn31 pn 331200 Jun 11 19:41 ecmwf_surface.grib
-rw-r--r-- 1 pn31 pn   7207 Jun 27 15:58 extr_mars_0627.ipynb
-rw-r--r-- 1 pn31 pn  36948 Jul  1 10:15 flujo_basico_C.jpg
-rw-r--r-- 1 pn31 pn  39713 Jul  1 10:10 flujo_basico_C.png
-rw-r--r-- 1 pn31 pn  42117 Jul  1 10:15 flujo_basico_python.jpg
-rw-r--r-- 1 pn31 pn  43265 Jul  1 10:10 flujo_basico_python.png
-rw-r--r-- 1 pn31 pn   6504 Jun 27 06:28 Introduccion_fuentes.ipynb
-rw-r--r-- 1 pn31 pn  83491 Jun 10 17:16 matroshka_grib_60.jpg
-rw-r--r-- 1 pn31 pn 118943 Jul  1 10:15 Modulos_para_grib.ipynb
-rw-r--r-- 1 pn31 pn     23 Jun 11 20:06 README.md


![flujo basico C](flujo_basico_C.png)

![flujo_basico_python](flujo_basico_python.png)

https://www.ecmwf.int/en/newsletter/169/computing/advanced-regridding-metview

https://metview.readthedocs.io/en/latest/

## Módulo xarray

## Módulo climetlab